In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import chardet


def detect_encoding(file_path):
    # First, read a small portion of the file to detect its encoding
    with open(file_path, 'rb') as f:
        raw_data = f.read(1024)  # Read the first 1024 bytes to detect encoding
    result = chardet.detect(raw_data)
    return result['encoding']

def read_file_with_fallback(file_path):
    # Try to open the file with the detected encoding; if that fails, try common encodings
    try:
        encoding = detect_encoding(file_path)
        with open(file_path, 'r', encoding=encoding) as f:
            return f.read()
    except UnicodeDecodeError:
        # Try common encodings
        encodings = ['utf-8', 'gbk', 'gb2312', 'gb18030', 'big5', 'latin1']
        for enc in encodings:
            try:
                with open(file_path, 'r', encoding=enc) as f:
                    return f.read()
            except UnicodeDecodeError:
                continue
        raise ValueError(f"Unable to decode file {file_path}; tried multiple encodings.")

def merge_files():
    # Open a file dialog to select multiple txt files
    file_paths = filedialog.askopenfilenames(filetypes=[("Text Files", "*.txt")])
    if len(file_paths) < 2:
        messagebox.showwarning("Warning", "At least two txt files must be selected.")
        return

    # Read and merge the file contents
    try:
        # Read the content of the first file
        merged_content = read_file_with_fallback(file_paths[0])

        # Read and merge the rest of the files (skipping the header line)
        for file_path in file_paths[1:]:
            content = read_file_with_fallback(file_path)
            lines = content.splitlines()[1:]  # Split into lines and remove the header line
            merged_content += '\n' + '\n'.join(lines)

        # Open a save file dialog
        save_path = filedialog.asksaveasfilename(
            defaultextension=".txt",
            filetypes=[("Text Files", "*.txt")],
            title="Save Merged File"
        )
        if not save_path:
            messagebox.showwarning("Warning", "File not saved, operation canceled.")
            return

        # Write the merged content to the file
        with open(save_path, 'w', encoding='utf-8') as merged_file:
            merged_file.write(merged_content)

        messagebox.showinfo("Completed", f"File has been successfully saved to:\n{save_path}")

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while merging files:\n{str(e)}")

# Create the main window
root = tk.Tk()
root.title("File Merging Program")

# Create a button to trigger the file merging process
merge_button = tk.Button(root, text="Select Files and Merge", command=merge_files)
merge_button.pack(padx=20, pady=20)

# Run the main loop
root.mainloop()
